# 目的
- Faster-RCNNの学習コードのテスト
- [github](https://github.com/chainer/chainercv/blob/master/examples/faster_rcnn/train.py)のコードのコピー

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import chainer

#from chainer.datasets import ConcatenatedDataset
from chainer.datasets import TransformDataset
from chainer import training
from chainer.training import extensions
from chainer.training.triggers import ManualScheduleTrigger

from chainercv.datasets import voc_bbox_label_names
from chainercv.datasets import VOCBboxDataset
from chainercv.extensions import DetectionVOCEvaluator
from chainercv.links import FasterRCNNVGG16
from chainercv.links.model.faster_rcnn import FasterRCNNTrainChain
from chainercv import transforms

/home/tokita/.pyenv/versions/anaconda2-4.4.0/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 実装
## BoundingBox DataSet

In [3]:
train_data = VOCBboxDataset(split='trainval', year='2007') # train + validation
test_data = VOCBboxDataset(split='test', year='2007',
                           use_difficult=True, return_difficult=True)

In [4]:
print(train_data.keys)
print(test_data.keys)

('img', 'bbox', 'label')
('img', 'bbox', 'label', 'difficult')


In [5]:
print(len(train_data.ids))
print(len(test_data.ids))

5011
4952


In [6]:
test_data[1][0].shape

(3, 500, 335)

## FasterRCNNモデルのロード

In [6]:
faster_rcnn = FasterRCNNVGG16(n_fg_class=len(voc_bbox_label_names), 
                              pretrained_model='imagenet')

In [7]:
# プリセットされた閾値を使用する
faster_rcnn.use_preset('evaluate')

## 学習用のプリセットされたChainをセット

In [8]:
gpu_id = 1

In [9]:
model = FasterRCNNTrainChain(faster_rcnn)
chainer.cuda.get_device_from_id(gpu_id).use()
model.to_gpu()

## Optimizer

In [10]:
optimizer = chainer.optimizers.MomentumSGD(lr=0.001, momentum=0.9)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer_hooks.WeightDecay(rate=0.0005))

## データのセット

In [11]:
class Transform(object):

    def __init__(self, faster_rcnn):
        self.faster_rcnn = faster_rcnn

    def __call__(self, in_data):
        img, bbox, label = in_data
        _, H, W = img.shape
        img = self.faster_rcnn.prepare(img)
        _, o_H, o_W = img.shape
        scale = o_H / H
        bbox = transforms.resize_bbox(bbox, (H, W), (o_H, o_W))

        # horizontally flip
        img, params = transforms.random_flip(
            img, x_random=True, return_param=True)
        bbox = transforms.flip_bbox(
            bbox, (o_H, o_W), x_flip=params['x_flip'])

        return img, bbox, label, scale

In [12]:
train_data = TransformDataset(train_data, Transform(faster_rcnn))

In [13]:
train_iter = chainer.iterators.MultiprocessIterator(
    train_data, batch_size=1, n_processes=None, shared_mem=100000000)
test_iter = chainer.iterators.SerialIterator(
    test_data, batch_size=1, repeat=False, shuffle=False)
updater = chainer.training.updaters.StandardUpdater(
    train_iter, optimizer, device=gpu_id)

In [14]:
n_iter = 70000
out_dir = './out'
trainer = training.Trainer(
    updater, (n_iter, 'iteration'), out=out_dir)

In [15]:
step_size = 50000

trainer.extend(
    extensions.snapshot_object(model.faster_rcnn, 'snapshot_model.npz'),
    trigger=(n_iter, 'iteration'))
trainer.extend(extensions.ExponentialShift('lr', 0.1),
               trigger=(step_size, 'iteration'))


In [16]:
log_interval = 20, 'iteration'
plot_interval = 3000, 'iteration'
print_interval = 20, 'iteration'

In [17]:
trainer.extend(chainer.training.extensions.observe_lr(),
               trigger=log_interval)
trainer.extend(extensions.LogReport(trigger=log_interval))
trainer.extend(extensions.PrintReport(
    ['iteration', 'epoch', 'elapsed_time', 'lr',
     'main/loss',
     'main/roi_loc_loss',
     'main/roi_cls_loss',
     'main/rpn_loc_loss',
     'main/rpn_cls_loss',
     'validation/main/map',
    ]), trigger=print_interval)
trainer.extend(extensions.ProgressBar(update_interval=10))

In [18]:
trainer.extend(
    extensions.PlotReport(
        ['main/loss'],
        file_name='loss.png', trigger=plot_interval
    ),
    trigger=plot_interval
)

In [19]:
trainer.extend(
    DetectionVOCEvaluator(
        test_iter, model.faster_rcnn, use_07_metric=True,
        label_names=voc_bbox_label_names),
    trigger=ManualScheduleTrigger(
        [step_size, n_iter], 'iteration'))

trainer.extend(extensions.dump_graph('main/loss'))

In [20]:
trainer.run()

     total [..................................................]  0.01%
this epoch [..................................................]  0.20%
        10 iter, 0 epoch / 70000 iterations
       inf iters/sec. Estimated time to finish: 0:00:00.
iteration   epoch       elapsed_time  lr          main/loss   main/roi_loc_loss  main/roi_cls_loss  main/rpn_loc_loss  main/rpn_cls_loss  validation/main/map
20          0           7.73568       0.001       2.31725     0.388415           1.20407            0.145397           0.579364                                
     total [..................................................]  0.03%
this epoch [..................................................]  0.40%
        20 iter, 0 epoch / 70000 iterations
    3.3313 iters/sec. Estimated time to finish: 5:50:06.837079.
     total [..................................................]  0.04%
this epoch [..................................................]  0.60%
        30 iter, 0 epoch / 70000 iterations
   

     total [..................................................]  0.36%
this epoch [##................................................]  4.99%
       250 iter, 0 epoch / 70000 iterations
    3.5791 iters/sec. Estimated time to finish: 5:24:47.965270.
260         0           74.769        0.001       1.33593     0.34945            0.593094           0.0899503          0.303435                                
     total [..................................................]  0.37%
this epoch [##................................................]  5.19%
       260 iter, 0 epoch / 70000 iterations
    3.5696 iters/sec. Estimated time to finish: 5:25:37.255533.
     total [..................................................]  0.39%
this epoch [##................................................]  5.39%
       270 iter, 0 epoch / 70000 iterations
    3.5817 iters/sec. Estimated time to finish: 5:24:28.282736.
280         0           80.2323       0.001       1.4005      0.303284           0.621207 

500         0           142.754       0.001       1.31329     0.349495           0.571897           0.102665           0.289233                                
     total [..................................................]  0.71%
this epoch [####..............................................]  9.98%
       500 iter, 0 epoch / 70000 iterations
    3.5502 iters/sec. Estimated time to finish: 5:26:16.426371.
     total [..................................................]  0.73%
this epoch [#####.............................................] 10.18%
       510 iter, 0 epoch / 70000 iterations
    3.5528 iters/sec. Estimated time to finish: 5:25:59.273080.
520         0           148.516       0.001       1.05454     0.341956           0.473152           0.0741312          0.165298                                
     total [..................................................]  0.74%
this epoch [#####.............................................] 10.38%
       520 iter, 0 epoch / 70000 itera

     total [..................................................]  1.07%
this epoch [#######...........................................] 14.97%
       750 iter, 0 epoch / 70000 iterations
    3.5301 iters/sec. Estimated time to finish: 5:26:57.190413.
760         0           217.407       0.001       1.08176     0.417238           0.429725           0.0647015          0.170092                                
     total [..................................................]  1.09%
this epoch [#######...........................................] 15.17%
       760 iter, 0 epoch / 70000 iterations
    3.5265 iters/sec. Estimated time to finish: 5:27:14.091306.
     total [..................................................]  1.10%
this epoch [#######...........................................] 15.37%
       770 iter, 0 epoch / 70000 iterations
    3.5289 iters/sec. Estimated time to finish: 5:26:57.735297.
780         0           223.157       0.001       1.08305     0.354887           0.510106 

1000        0           287.141       0.001       1.02131     0.365016           0.417052           0.0674125          0.171829                                
     total [..................................................]  1.43%
this epoch [#########.........................................] 19.96%
      1000 iter, 0 epoch / 70000 iterations
    3.5055 iters/sec. Estimated time to finish: 5:28:03.183515.
     total [..................................................]  1.44%
this epoch [##########........................................] 20.16%
      1010 iter, 0 epoch / 70000 iterations
    3.5084 iters/sec. Estimated time to finish: 5:27:44.253763.
1020        0           292.872       0.001       1.18297     0.412503           0.51718            0.0557109          0.197578                                
     total [..................................................]  1.46%
this epoch [##########........................................] 20.36%
      1020 iter, 0 epoch / 70000 itera

...

69000       13          20563.4       0.0001      0.273873    0.118183           0.0829349          0.0328481          0.0399063                               
     total [#################################################.] 98.57%
this epoch [######################################............] 76.97%
     69000 iter, 13 epoch / 70000 iterations
    3.4192 iters/sec. Estimated time to finish: 0:04:52.466009.
     total [#################################################.] 98.59%
this epoch [######################################............] 77.17%
     69010 iter, 13 epoch / 70000 iterations
    3.4165 iters/sec. Estimated time to finish: 0:04:49.766891.
69020       13          20569.4       0.0001      0.340755    0.13149            0.0842693          0.0675319          0.0574638                               
     total [#################################################.] 98.60%
this epoch [######################################............] 77.37%
     69020 iter, 13 epoch / 70000 it

     total [#################################################.] 98.93%
this epoch [########################################..........] 81.96%
     69250 iter, 13 epoch / 70000 iterations
    3.4153 iters/sec. Estimated time to finish: 0:03:39.600132.
69260       13          20639.7       0.0001      0.336692    0.135808           0.0884181          0.0680965          0.0443696                               
     total [#################################################.] 98.94%
this epoch [#########################################.........] 82.16%
     69260 iter, 13 epoch / 70000 iterations
    3.4149 iters/sec. Estimated time to finish: 0:03:36.700062.
     total [#################################################.] 98.96%
this epoch [#########################################.........] 82.36%
     69270 iter, 13 epoch / 70000 iterations
    3.4137 iters/sec. Estimated time to finish: 0:03:33.847107.
69280       13          20645.6       0.0001      0.266393    0.127103           0.0749

69500       13          20710.3       0.0001      0.322867    0.129521           0.105802           0.048221           0.0393241                               
     total [#################################################.] 99.29%
this epoch [###########################################.......] 86.95%
     69500 iter, 13 epoch / 70000 iterations
    3.4089 iters/sec. Estimated time to finish: 0:02:26.674726.
     total [#################################################.] 99.30%
this epoch [###########################################.......] 87.15%
     69510 iter, 13 epoch / 70000 iterations
    3.4084 iters/sec. Estimated time to finish: 0:02:23.761279.
69520       13          20716.2       0.0001      0.243347    0.115087           0.0750905          0.0225945          0.030575                                
     total [#################################################.] 99.31%
this epoch [###########################################.......] 87.35%
     69520 iter, 13 epoch / 70000 it

     total [#################################################.] 99.64%
this epoch [#############################################.....] 91.94%
     69750 iter, 13 epoch / 70000 iterations
    3.4005 iters/sec. Estimated time to finish: 0:01:13.518859.
69760       13          20787         0.0001      0.206438    0.0908536          0.0704925          0.0244366          0.0206552                               
     total [#################################################.] 99.66%
this epoch [##############################################....] 92.14%
     69760 iter, 13 epoch / 70000 iterations
    3.3982 iters/sec. Estimated time to finish: 0:01:10.625334.
     total [#################################################.] 99.67%
this epoch [##############################################....] 92.34%
     69770 iter, 13 epoch / 70000 iterations
    3.3969 iters/sec. Estimated time to finish: 0:01:07.709027.
69780       13          20793         0.0001      0.226094    0.0890014          0.0739

70000       13          21489         0.0001      0.304343    0.121913           0.0976115          0.0422373          0.0425806          0.69941              
     total [##################################################] 100.00%
this epoch [################################################..] 96.93%
     70000 iter, 13 epoch / 70000 iterations
    1.0804 iters/sec. Estimated time to finish: 0:00:00.
